# Projeto 1 - Ciência dos Dados

Nome: Camila Soares Magni

Nome: Lincoln Rodrigo Pereira Melo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

### Etapas do desenvolvimento do projeto:

Primeiramente, 

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Lincoln\Desktop\Matérias2021.2\Cdados\LAST\Proj1Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'airfryer.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,o twitter vai me fazer comprar uma airfryer,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"quero assar pão de queijo, mas o forno daqui d...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"bom, aparentemente deu ruim e saiu toda a tint...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,de airfryer a panelas: polishop celebra 22 ano...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,@danielsmarconn mano eu tô louca nessa airfrye...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,"o bom da airfryer fica tudo pronto rápido, não...",1
1,um app pra calcular se é mais barato fazer a c...,0
2,@raytotsukishiro aumentou o preço de tudo. aqu...,0
3,meti um pão de queijo na airfryer 😌,1
4,"@alcysio @safbf pô alcysio, me adaptei total p...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos a fritadeira sem óleo airfryer como nosso produto. Para a classificação dos tweets, consideramos todas as mensagens que expressam alguma opinião sobre o produto, sejam elogios , críticas e receitas como relevantes, assim como as que falam sobre um desejo de adquirir uma airfryer. O resto foi classificado como irrelevante, incluindo mensagens sobre outros assuntos que não fazem sentido para a análise do produto(Exemplo: "Parece que estou dentro de uma airfryer"). 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
import pandas as pd
import numpy as np
from IPython.display import display
import re 
import nltk

# Função de limpeza dos textos
def cleanup(text):
    punctuation = '[""''!-.:?;“”~1234567890]'                            # sinais que queremos retirar do texto
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)                              # retira os sinais
    text_subbed = re.sub(r"http\S+", "", text_subbed)                    # retira os URL
    text_subbed = re.sub('@[^\s]+','',text_subbed,flags=re.MULTILINE)    # retira os nomes de usuário
    text_subbed = re.sub(r"kk\S+", "kk", text_subbed)                    # transformar as strings que começam com kk em 'kk'
    text_subbed = re.sub(r"compr\S+", "compr", text_subbed) 
    text_subbed = re.sub(r"limp\S+", "limp", text_subbed) 
    text_subbed = re.sub(r"aa\S+", "", text_subbed)
    text_subbed = re.sub(r"airfryer\S+", "airfryer", text_subbed)
    text_subbed = re.sub(r"queij\S+", "queij", text_subbed)
    text_subbed = re.sub(r"batat\S+", "batata", text_subbed) 
    aux = nltk.TweetTokenizer()
    text_subbed = aux.tokenize(text_subbed)                              # separa os emojis
    string_final = ' '.join(text_subbed)                                 # string limpa
    return string_final

In [7]:
# Função para tirar as 'Stopwords' em português:
# nltk.download('stopwords') - INSTALAR, caso vc não tenha instalado
def retirar_stopwords(lista):
    lista_limpa = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for palavra in lista:
        if palavra not in stopwords:
            lista_limpa.append(palavra)
    return lista_limpa

In [8]:
lista = []
for linha in train['Treinamento']:
    limpo = cleanup(linha.lower())             # Limpa a planilha Treinamento e tira as letras maiúsculas 
    a = limpo.split()
    b = retirar_stopwords(a)
    limpo = ' '.join(b)  
    lista.append(limpo)
    
serie_train = pd.Series(lista)                 # Transforma a coluna da planilha Treinamento em Series
train['Treinamento'] = serie_train        # substitui os tweets originais da planilha, pelos tweets limpos
train.head()

,Treinamento,Classificação,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,twitter vai fazer compr airfryer,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,quero assar pão queij forno daqui casa bosta p...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bom aparentemente deu ruim saiu toda tinta / t...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,airfryer panelas polishop celebra anos descont...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mano tô louca nessa airfryer serase boa,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Função Classificadora Naive-Bayes

In [9]:
def classificador(dataframe, series):                         #recebe o dataframe e a serie de tweets que será classificada
    
    filtro = train.Classificação==1          # filtra os tweets classificados como relevantes
    relevantes = train.loc[filtro, :]        # dataframe com tweets relevantes

    filtro2 = train.Classificação==0        # filtra os tweets classificados como irrelevantes
    irrelevantes = train.loc[filtro2, :]    # dataframe com tweets irrelevantes

    # Transforma o dataframe relevantes em um único texto
    string_relevante = ''
    for linha in relevantes.iloc[:,0]:
        string_relevante+= ' '+ linha

    # Transforma o dataframe irrelevantes em um único texto
    string_irrelevante = ''
    for linha in irrelevantes.iloc[:,0]:
        string_irrelevante+= ' '+ linha

    todas_relevantes = string_relevante.split()        #Lista com todas as palavras que aparece na string relevante
    todas_irrelevantes = string_irrelevante.split()    #Lista com todas as palavras que aparece na string irrelevante

    serie_relevante = pd.Series(todas_relevantes)      #Transforma a lista de strings relevantes em um series
    serie_irrelevante = pd.Series(todas_irrelevantes)  #Transforma a lista de strings irrelevantes em um series

    todas = string_irrelevante + string_relevante          # juntando todas as palavras que aparecem na base de dados Treinamento
    todas_as_palavras = todas.split()                      # lista com todas as palavras
    serie_total = pd.Series(todas_as_palavras) 
    serie_total

    P_R = len(todas_relevantes)/len(todas_as_palavras)        # probabilidade de uma palavra ser relevante
    P_I = len(todas_irrelevantes)/len(todas_as_palavras)      # probabilidade de uma palavra ser irrelevante

    freq_rel_absoluta = serie_relevante.value_counts()           # quantidade de cada palavra na serie relevante
    freq_irrel_absoluta = serie_irrelevante.value_counts()       # quantidade de cada palavra na serie irrelevante
    freq_tot_absoluta = serie_total.value_counts()               # quantidade de cada palavra na serie total


    
    classificados = []
    for frase in series:
        lista_palavras = frase.split()
        lista_palavras = retirar_stopwords(lista_palavras)    #implementa função que tira as stopwords
        k,i = 0,0                                             #contador de palavras NÃO presentes em relevantes (k) e em irrevelantes (i)
        for palavra in lista_palavras:
            if palavra not in todas_relevantes: 
                k += 1
            if palavra not in todas_irrelevantes:
                i += 1

        prob_frase_dado_relevante = 1
        prob_frase_dado_irrelevante = 1

        if k == 0:                                   #se tiver palavras no series que não estao na nossa base de dados relevantes
            for palavra in lista_palavras:
                prob = freq_rel_absoluta[palavra] / freq_rel_absoluta.sum()
                prob_frase_dado_relevante *= prob           #multiplica a probabilidade de cada palavra dado frase

        else:
            for palavra in lista_palavras:
                if palavra in freq_rel_absoluta:
                    prob = (freq_rel_absoluta[palavra]+ 1 ) / (freq_rel_absoluta.sum()+ len(freq_tot_absoluta)) # SUAVIZAÇÃO DE LAPLACE"
                else: 
                    prob = (0 + 1 ) / (freq_rel_absoluta.sum()+ len(freq_tot_absoluta))
                prob_frase_dado_relevante *= prob           #multiplica a probabilidade de cada palavra dado frase
        
        if i == 0:
            for palavra in lista_palavras:
                prob = ( freq_irrel_absoluta[palavra] ) / (freq_irrel_absoluta.sum())
                prob_frase_dado_irrelevante *= prob     

        else:
            for palavra in lista_palavras:
                if palavra in freq_irrel_absoluta:
                    prob = ( freq_irrel_absoluta[palavra]+ 1 ) / (freq_irrel_absoluta.sum() +len(freq_tot_absoluta))
                else:
                    prob = ( 0 + 1 ) / (freq_irrel_absoluta.sum() +len(freq_tot_absoluta))
                prob_frase_dado_irrelevante *= prob         #multiplica a probabilidade de cada palavra dado frase
                #multiplica a probabilidade de cada palavra dado frase
                
        probRdadoFrase = prob_frase_dado_relevante* P_R
        probIdadoFrase = prob_frase_dado_irrelevante* P_I
        
        if probRdadoFrase > probIdadoFrase:
            classificados.append(1)                         #classifica como relevante
        else:  
            classificados.append(0)                         #classifica como irrelevante
    
    serie_classificado = pd.Series(classificados)
    novo_df = dataframe.iloc[:, [0,1]]
    novo_df['Classificado'] = serie_classificado
    return novo_df
                

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [10]:
test

,Teste,Classificação
0,"o bom da airfryer fica tudo pronto rápido, não...",1
1,um app pra calcular se é mais barato fazer a c...,0
2,@raytotsukishiro aumentou o preço de tudo. aqu...,0
3,meti um pão de queijo na airfryer 😌,1
4,"@alcysio @safbf pô alcysio, me adaptei total p...",1
...,...,...
195,tutorial bolinha de queijo✨\n\n200g de queijo ...,1
196,pessoal daqui de casa inventou de querer fazer...,1
197,"eu nao sei voces, mas eu ganhei uma airfryer g...",1
198,@ibebelly amanhã eu vou fazer um pão de alho d...,1


In [11]:
lista = []

for linha in test['Teste']:
    limpo = cleanup(linha.lower())             # Limpa a planilha Teste e tira as letras maiúsculas
    a = limpo.split()
    b = retirar_stopwords(a)
    limpo = ' '.join(b)  
    lista.append(limpo)

test['Teste'] = pd.Series(lista)             # Substitui os tweets originais da planilha, pelos tweets limpos
test.head()

,Teste,Classificação
0,bom airfryer fica tudo pronto rápido quero out...,1
1,app pra calcular barato fazer comida airfryer ...,0
2,aumentou preço tudo aqui gente compr airfryer ...,0
3,meti pão queij airfryer 😌,1
4,pô alcysio adaptei total pra usar mínimo gás a...,1


# Implementando a função...

In [12]:
df = test               # dataframe que será argumento da função classificadora
series= test['Teste']   # series que será argumento da função classificadora

In [13]:
# Implementa função
df = classificador(df, series) #dataframe que passou pelo função para classificar os tweets
df.head()

,Teste,Classificação,Classificado
0,bom airfryer fica tudo pronto rápido quero out...,1,1
1,app pra calcular barato fazer comida airfryer ...,0,1
2,aumentou preço tudo aqui gente compr airfryer ...,0,0
3,meti pão queij airfryer 😌,1,1
4,pô alcysio adaptei total pra usar mínimo gás a...,1,1


## Extraindo as contagens para verificação de performance

In [14]:
tabela = pd.crosstab(df.Classificado, df.Classificação, normalize= True)  # tabela de porcentagens
tabela

Classificação,0,1
Classificado,,
0,0.28,0.095
1,0.17,0.455


Porcentagem de verdadeiros positivos (mensagens relevantes e que são
classificadas como relevantes)

In [15]:
verdadeiros_positivos = tabela.iloc[1,1]*100
verdadeiros_positivos

45.5

Porcentagem de falsos positivos (mensagens irrelevantes e que são classificadas
como relevantes)

In [16]:
falsos_positivos = tabela.iloc[1,0]*100
falsos_positivos

17.0

Porcentagem de verdadeiros negativos (mensagens irrelevantes e que são
classificadas como irrelevantes)

In [17]:
verdadeiros_negativos = tabela.iloc[0,0]*100
verdadeiros_negativos

28.000000000000004

Porcentagem de falsos negativos (mensagens relevantes e que são classificadas
como irrelevantes)

In [18]:
falsos_negativos = tabela.iloc[0,1]*100
falsos_negativos

9.5

Acurácia (mensagens corretamente classificadas, independente da categoria)

In [19]:
acuracia = verdadeiros_positivos + verdadeiros_negativos
acuracia

73.5

___
### Concluindo

#### Performance do classificador e possíveis melhorias  

Comparativo qualitativo sobre os percentuais obtidos:

A partir dos percentuais de verdadeiros positivos e negativos obtidos, observa-se que o nosso classificador possui acurácia de 73.5%, o que significa que dos 100% dos tweets, teríamos 26.5% de falhas (falsos positivos e negativos).

Analisando de maneira mais aprofundada todos os tweets classificados como relevantes (62.5% do total), temos que a porcentagem de verdadeiros positivos (45.5%) é mais que o dobro do que a de falsos positivos (17%). Em outras palavras, 72.8% dos tweets classificados como relevantes, foram classificados corretamente. 

Da mesma forma, olhando para os tweets classificados como irrelevantes (37.5% do total, temos 28% de acertos, ou seja, 74,6% dos tweets classificados como irrelevantes.

As mensagens com sarcasmo ou dupla negação são tratadas pelo classificador da mesma forma que qualquer outra mensagem, isto é, pela análise das probabilidades de cada palavra do tweet. Então, algumas são classificadas corretamente e outras, não, já que muitas palavras aparecem na Base de Treinamento como relevantes. 

Dessa forma, levando em consideração os percentuais obtidos, consideramos que a performance do nosso classificador está aceitável, porém há algumas melhorias que poderiam ser implementadas, como .....





#### Por que continuar financiando o projeto? 

O projeto deve continuar sendo financiado, porque a partir dele, é possível entender os interesses e as opiniões do público alvo da empresa, assim como elogios e críticas ao produto, para que seja possível implementar melhorias em pontos com maiores reclamações.

#### Por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets?

Se nós alimentarmos a base de Treinamento utilizando o próprio classificador, o percentual de erros aumentaria significativamente. Isso porque, os erros do classificador se propagariam para a classificação da base Treinamento, que por sua vez, alimentaria a classificação de novas bases, propagando cada vez mais os erros

#### Diferentes cenários de uso para o classificador Naive-Bayes 

O classificador Naive-Bayes pode ser aplicado em diversas situações diferentes, como no diagnostico de doenças ....

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [20]:
test = pd.read_excel(filename, sheet_name = "Teste" )
test_novo_tudo = test.rename(columns={ "Teste" : "Tudo"})

train = pd.read_excel(filename,sheet_name = "Treinamento")
train_novo_tudo = train.rename(columns={ "Treinamento" : "Tudo" })

df_tudo = pd.concat([train_novo_tudo,test_novo_tudo])
df_tudo

,Tudo,Classificação,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,o twitter vai me fazer comprar uma airfryer,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"quero assar pão de queijo, mas o forno daqui d...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"bom, aparentemente deu ruim e saiu toda a tint...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,de airfryer a panelas: polishop celebra 22 ano...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,@danielsmarconn mano eu tô louca nessa airfrye...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
195,tutorial bolinha de queijo✨\n\n200g de queijo ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,pessoal daqui de casa inventou de querer fazer...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,"eu nao sei voces, mas eu ganhei uma airfryer g...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,@ibebelly amanhã eu vou fazer um pão de alho d...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
lista = []

for linha in df_tudo['Tudo']:
    limpo = cleanup(linha.lower())             # Limpa a planilha Treinamento e tira as letras maiúsculas 
    a = limpo.split()
    b = retirar_stopwords(a)
    limpo = ' '.join(b) 
    lista.append(limpo)
    
serie = pd.Series(lista)                 # Transforma a coluna da planilha Treinamento em Series
df_tudo['Tudo'] = serie

df_tudo.head()

,Tudo,Classificação,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,twitter vai fazer compr airfryer,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,quero assar pão queij forno daqui casa bosta p...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bom aparentemente deu ruim saiu toda tinta / t...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,airfryer panelas polishop celebra anos descont...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mano tô louca nessa airfryer serase boa,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
from sklearn.model_selection import train_test_split
def separa_dataframe(df):
    train, test_new = train_test_split(df_tudo, test_size=200)
    return train, test_new

In [23]:
lista_da_acuracia = []

for i in range(100):
    
    datas = separa_dataframe(df_tudo)
    train = datas[0]
    test_new = datas[1]
    df = classificador(test_new, test_new["Tudo"])
    
    tabela_nova = pd.crosstab(df.Classificado, df.Classificação, normalize= True)
    acuracia = tabela_nova.iloc[1,1]*100 + tabela_nova.iloc[0,0]*100
    lista_da_acuracia.append(acuracia)
    
print(lista_da_acuracia)

<ipython-input-9-5b1dfab55f08>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo_df['Classificado'] = serie_classificado
<ipython-input-9-5b1dfab55f08>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo_df['Classificado'] = serie_classificado
<ipython-input-9-5b1dfab55f08>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

[50.0, 49.69325153374233, 46.2962962962963, 53.164556962025316, 56.44171779141104, 48.07692307692308, 41.139240506329116, 57.5, 52.147239263803684, 50.3030303030303, 48.76543209876543, 50.61728395061728, 52.40963855421687, 45.50898203592814, 49.38271604938272, 50.61728395061728, 54.60526315789473, 51.973684210526315, 52.53164556962025, 53.164556962025316, 52.53164556962025, 56.09756097560975, 51.23456790123457, 47.53086419753086, 55.27950310559007, 51.2987012987013, 50.0, 56.96969696969697, 52.17391304347826, 50.59523809523809, 50.0, 49.342105263157904, 47.5, 48.701298701298704, 55.15151515151515, 54.43037974683544, 49.36708860759494, 58.38509316770186, 50.943396226415096, 40.60606060606061, 56.36363636363636, 41.05960264900662, 57.668711656441715, 42.85714285714286, 53.37423312883435, 51.265822784810126, 45.28301886792453, 49.681528662420384, 50.0, 49.382716049382715, 49.079754601226995, 52.97619047619048, 55.9748427672956, 55.4140127388535, 39.87730061349693, 45.22292993630573, 51.87

<ipython-input-9-5b1dfab55f08>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo_df['Classificado'] = serie_classificado


### Histograma com percentuais de acertos

##### Vantagens ou desvantagens sobre construir um classificador considerando uma única vez a divisão da base de dados em treinamento e em teste:


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**